In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
batch_size = 32
num_workers = 4
lr = 0.001
num_epochs = 200

In [3]:
conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 每经过一个 VGGBlock，HW 减半，224 / 32 = 7
fc_features = 512 * 7 * 7 # c * w * h
fc_hidden_units = 4096

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

In [5]:
def vgg_block(num_blocks, in_channels, out_channels):
    blk = []
    blk.append(nn.Conv2d(in_channels, out_channels, 3, 1, 1))
    blk.append(nn.ReLU())
    for i in range(num_blocks - 1):
        blk.append(nn.Conv2d(out_channels, out_channels, 3, 1, 1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(2, 2))
    return nn.Sequential(*blk)

In [6]:
class VGGNet(nn.Module):
    def __init__(self, conv_arch, fc_features, fc_hidden_units):
        super(VGGNet, self).__init__()
        self.conv = nn.Sequential()
        for i, (num_blocks, in_channels, out_channels) in enumerate(conv_arch):
            self.conv.add_module(f'vgg_block_{i}', vgg_block(num_blocks, in_channels, out_channels))
        self.fc = nn.Sequential(
            nn.Linear(fc_features, fc_hidden_units),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(fc_hidden_units, fc_hidden_units),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(fc_hidden_units, 10))
    
    def forward(self, x):
        b, c, h, w = x.shape
        feature = self.conv(x)
        return self.fc(feature.view(b, -1))

In [7]:
trans = []
trans.append(transforms.Resize((224, 224)))
trans.append(transforms.ToTensor())
transform = transforms.Compose(trans)
mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=True, download=True, transform=transform)
mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST', train=False, download=True, transform=transform)
print(len(mnist_train), len(mnist_test))
train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=num_workers)

60000 10000


In [8]:
net = VGGNet(conv_arch, fc_features, fc_hidden_units).cuda()
optimizer = torch.optim.Adam(net.parameters(), lr)
loss = torch.nn.CrossEntropyLoss()

In [9]:
def train_FashionMNIST(net, train_iter, optimizer):
    train_loss = 0.0
    train_acc = 0.0
    train_num = 0
    
    for X, y in train_iter:
        X = X.cuda()
        y = y.cuda()
        y_hat = net(X)
        l = loss(y_hat, y)
        optimizer.zero_grad()
        # net.zero_grad()
        l.backward()
        optimizer.step()
        
        train_loss += l.item()
        train_acc += (y_hat.argmax(dim=1) == y).sum().item()
        train_num += y.shape[0]
    
    train_loss /= train_num
    train_acc /= train_num
    print('train loss: %.4f, train acc: %.3f' % (train_loss, train_acc))

In [10]:
def test_FashionMNIST(net, test_iter):
    test_acc = 0.0
    test_num = 0
    
    for X, y in test_iter:
        X = X.cuda()
        y = y.cuda()
        y_hat = net(X)
        test_acc += (y_hat.argmax(dim=1) == y).sum().item()
        test_num += y.shape[0]
    
    test_acc /= test_num
    print('test acc: %.3f' % (test_acc))

In [ ]:
for i in range(num_epochs):
    print(f'epoch: {i}')
    train_FashionMNIST(net, train_iter, optimizer)
    test_FashionMNIST(net, test_iter)
    print('----------------')

epoch: 0


In [ ]:
# Adam: lr=0.001, epoch x, test_acc=x, batch_size=64